In [ ]:
##### EL CÓDIGO CONSISTE DE DOS PARTES:
##### 1. OBTENER EL MOVIMIENTO DIARIO DE LAS ACCIONES 
##### BAJO EL TÍTULO: "WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL"
##### 2. OBTENER LA INFORMACIÓN FINANCIERA DE LAS EMPRESAS LISTADAS EN LA BVL (LAS QUE ESCOJA)
##### BAJO EL TÍTULO: "WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE UNA EMPRESA PERUANA DE LA BVL"
########## NOTA 1: DEBE TOMAR EN CUENTA QUE LAS PÁGINAS ESTÁN SUJETAS A CAMBIOS,
########## POR LO QUE EN EL FUTURO EL CÓDIGO PODRÍA REQUERIR CIERTAS VARIACIONES
########## NOTA 2: ALGUNOS PROCESOS TOMAN CIERTO TIEMPO PARA GARANTIZAR UNA CORRECTA EXTRACCIÓN DE DATOS.
########## NOTA 3: ESTE ES UN CÓDIGO LIBRE ( YA SEA COMO MATERIAL DE APRENDIZAJE O SI DESEA REALIZAR SU PROPIA VERSIÓN)
########## NOTA 4: SIEMPRE HAY ESPACIO PARA MEJORAS Y ACTUALIZACIONES
########## NOTA 5: ASEGÚRESE DE SEGUIR LAS INTRUCCIONES Y TOMAR EN CUENTA LOS COMENTARIOS

In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#Primero obtendremos el html de la tabla que contiene los datos
#Importando las librerías necesarias
#En caso no se tengan las librerías instalarlas
#pip install BeautifulSoup
#pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from csv import writer
import time
import openpyxl
import pandas as pd

# Inicializar el controlador de Selenium
driver = webdriver.Chrome('/path/to/chromedriver') # Reemplazar con la ruta al controlador de Chrome

# Abrir la página web
url = "https://www.bvl.com.pe/mercado/movimientos-diarios"
driver.get(url)

# Esperar a que cargue la página, 10 segundos es prudente
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-dark")))

# Obtener el HTML de la página
html = driver.page_source

# Analizar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla en el HTML
tabla = soup.find("table", {"class": "table-dark"})

# Imprimir la tabla
print(tabla)

# Cerrar el controlador de Selenium
driver.quit()

In [34]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#Obtenemos las filas dentro de la tabla y el número
filas = tabla.find_all('tr', {'class': 'bvl-row'})
#print(filas) #se puede imprimir el html en caso de querer revisarlo
num_filas=len(filas)-1
print(num_filas)
print(soup)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#Primero extraemos el encabezado y el número de columnas
valores=filas[0].find_all('th', {'class': 'bvl-header-cell'})
columnas=len(valores)-1
print(columnas)

In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#1era opción: Guardar los datos en un CSV
with open('Daily_Movements_BVL.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = []
    for columna in range(columnas):
        texto = values[columna].text.strip()
        header.append(texto)
    thewriter.writerow(header)                                                                                                                             
    for fila in range(num_filas-1):
        datos_fila = []  # Crea una lista vacía para los datos de la fila actual
        for columna in range(columnas):
            try:
                spans = filas[fila+1].find_all('span', {'class': 'ng-star-inserted'})
                texto = spans[columna].text.strip()
                texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
                datos_fila.append(texto)  # Agrega el valor de datos a la lista de datos de la fila actual
            except:
                tds = filas[fila+1].find_all('td', {'class': 'ng-star-inserted'})
                texto = tds[columna].text.strip()
                texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
                datos_fila.append(texto)  # Agrega el valor de datos a la lista de datos de la fila actual
        thewriter.writerow(datos_fila)  # Escribe la lista de datos de la fila actual al archivo CSV

In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#2da opción: Guardar los datos en un XLSX
# Crear un archivo de Excel nuevo
workbook = openpyxl.Workbook()

# Seleccionar la primera hoja
sheet = workbook.active

# Crear los encabezados
header = []
for columna in range(columnas):
    texto = values[columna].text.strip()
    header.append(texto)

# Agregar los encabezados a la primera fila de la hoja
sheet.append(header)

# Obtener los datos de las filas restantes y agregarlos al archivo de Excel
for fila in range(num_filas-1):
    fila_datos = []
    for columna in range(columnas):
        try:
            spans = filas[fila+1].find_all('span', {'class': 'ng-star-inserted'})
            texto = spans[columna].text.strip()
            texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
            fila_datos.append(texto)
        except:
            tds = filas[fila+1].find_all('td', {'class': 'ng-star-inserted'})
            texto = tds[columna].text.strip()
            texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
            fila_datos.append(texto)
    sheet.append(fila_datos)

# Guardar el archivo de Excel
workbook.save('Daily_Movements_BVL.xlsx')

In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#3era opción: Guardar los datos en un DataFrame
#Esta opción se puede facilitar al tener los anteriores documentos creados
#Solo sería necesario leerlos con la librería pandas

# Leer archivo CSV y crear DataFrame
#df_csv = pd.read_csv('Daily_Movements_BVL.csv')

# Leer archivo XLSX y crear DataFrame
#df_xlsx = pd.read_excel('Daily_Movements_BVL.xlsx')

#Caso desde cero
# Crear un DataFrame vacío con las columnas correspondientes
header = []
for columna in range(columnas):
    texto = values[columna].text.strip()
    header.append(texto)
df = pd.DataFrame(columns=header)

# Agregar los datos al DataFrame
data_frames = []
for fila in range(num_filas-1):
    datos_fila = {}  # Crea un diccionario vacío para los datos de la fila actual
    for columna in range(columnas):
        try:
            spans = filas[fila+1].find_all('span', {'class': 'ng-star-inserted'})
            texto = spans[columna].text.strip()
            texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
            datos_fila[header[columna]] = texto  # Agrega el valor de datos al diccionario de la fila actual
        except:
            tds = filas[fila+1].find_all('td', {'class': 'ng-star-inserted'})
            texto = tds[columna].text.strip()
            texto = texto.replace("S/", "").replace("US$", "").replace("%", "")
            datos_fila[header[columna]] = texto  # Agrega el valor de datos al diccionario de la fila actual
    data_frames.append(pd.DataFrame.from_dict(datos_fila, orient='index').T)
df = pd.concat(data_frames, ignore_index=True)  # Concatena todos los DataFrames individuales


# En caso que se quiera guardar el DataFrame en un archivo Excel
#df.to_excel('Dataframe_BVL.xlsx', index=False)

#Observar los primeros 5 datos
print(df.head())

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANA DE LA BVL ########

#Se obtienen los Estados Financieros de las empresas listadas en la BVL y que se elija
#Primero obtener los enlaces que contienen la información financiera

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from csv import writer
import time
import openpyxl
import pandas as pd

# Inicializar el controlador de Selenium
driver = webdriver.Chrome('/path/to/chromedriver') # Reemplazar con la ruta al controlador de Chrome

# Abrir la página web
url = "https://www.bvl.com.pe/mercado/movimientos-diarios"
driver.get(url)

# Esperar a que cargue la página, 10 segundos es prudente
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-dark")))

# Obtener el HTML de la página
html = driver.page_source

# Analizar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontrar la tabla en el HTML
tabla = soup.find("table", {"class": "table-dark"})

# Imprimir la tabla
#print(tabla)

#Obtenemos las filas dentro de la tabla y el número
time.sleep(5) # Pausa de 5 segundos
filas = tabla.find_all('tr', {'class': 'bvl-row'})
#print(filas) #se puede imprimir el html en caso de querer revisarlo
num_filas=len(filas)-1
#print(num_filas)

#Extraer el html donde se encuentran la información adicional de las acciones
extra_info = tabla.find_all('tr', {'class': 'details-row'})
#print(extra_info[0])


# Encontrar todos los elementos li que contienen un enlace con el título "Información financiera"
informacion_financiera = []
for fila in range(num_filas-1):
    li_list = extra_info[fila].find_all('li', {'class': 'ng-star-inserted'})
    time.sleep(2)
    for li in li_list:
        a = li.find('a', {'title': 'Información financiera'})
        time.sleep(2)
        if a is not None:
            informacion_financiera.append(a['href'])

# Imprimir los enlaces obtenidos
print(informacion_financiera)

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANAS DE LA BVL ########

#Enlazamos los links a la empresa correspondiente (nombre de la empresa y símbolo de la acción)
empresas = {}  # Diccionario vacío para guardar la información

for url in informacion_financiera:
    # Inicializar el controlador de Selenium
    driver = webdriver.Chrome('/path/to/chromedriver') # Reemplazar con la ruta al controlador de Chrome

    # Abrir la página web
    driver.get(url)

    # Esperar a que cargue la página, 10 segundos es prudente
    wait = WebDriverWait(driver, 10)

    # Obtener el HTML de la página
    html = driver.page_source

    # Analizar el HTML con BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Obtener el nombre de la empresa
    company_name = soup.find('h1', {'id': 'company-name'}).text.strip()
    #print(company_name)

    # Obtener el acrónimo de la acción
    div_element = soup.find('div', {'class': 'g-site-select--label g-site-select--label--large'})
    symbol = div_element.span.text

    #print(symbol)

    # Agregar la información al diccionario
    empresas[url] = {'nombre': company_name, 'simbolo': symbol}

    driver.quit() #Cerrar el navegador
    time.sleep(2)
    
print(empresas)

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANAS DE LA BVL ########

#Elija las empresas de la que desea buscar su Información Financiera
#Se obtienen los nombres, símbolo de la acción y los links
#Asegúrese de seguir las instrucciones
#Las listas (ya sea de nombres o símbolos) puedes utilizarlo para realizar la búsqueda
#Toma en cuenta que es una búsqueda exacta (referencia al anterior comentario)

#Creando la función de búsqueda
def buscar_empresas():
    empresas_seleccionadas = {}
    contador=1
    while True:
        opcion_busqueda = input("¿Desea buscar por nombre o por símbolo? Escriba N para buscar por nombre o S para buscar por símbolo: ")
        if opcion_busqueda.upper() == 'N':
            opcion_lista = input("¿Desea ver la lista de nombres de empresas? Escriba S para sí o N para no: ")
            if opcion_lista.upper() == 'S':
                print("Lista de empresas:")
                for empresa in sorted(empresas.values(), key=lambda x: x['nombre']):
                    print("- " + empresa['nombre'])
            nombre_empresa = input("Escriba el nombre de la empresa que desea buscar: ")
            empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['nombre'] == nombre_empresa.upper()]
            while len(empresas_encontradas) > 1:
                print("Se encontraron varias empresas con el mismo nombre, por favor indique el símbolo de la empresa que desea buscar:")
                opcion_lista_simbolos = input("¿Desea ver la lista de símbolos de empresas? Escriba S para sí o N para no: ")
                if opcion_lista_simbolos.upper() == 'S':
                    print("Lista de símbolos de empresas:")
                    for empresa in sorted(empresas.values(), key=lambda x: x['nombre']):
                        print("- " + empresa['nombre'])
                simbolo_empresa = input("Escriba el símbolo de la empresa que desea buscar: ")
                empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['nombre'] == nombre_empresa and datos_empresa['simbolo'] == simbolo_empresa.upper()]
            if len(empresas_encontradas) == 0:
                print("No se encontró la empresa.")
            else:
                print("Puedes encontrar información financiera de la empresa en el siguiente link:", empresas_encontradas[0])
                empresa_encontrada = empresas_encontradas[0]
                datos_empresa = empresas[empresa_encontrada]
                numero_empresa= empresas_encontradas[0]
                numero_empresa = 'empresa' + str(contador)
                datos_empresa_seleccionada = {'nombre': datos_empresa['nombre'], 'simbolo': datos_empresa['simbolo'], 'link':empresas_encontradas[0]}
                empresas_seleccionadas[numero_empresa] = datos_empresa_seleccionada
                contador += 1
        elif opcion_busqueda.upper() == 'S':
            opcion_lista = input("¿Desea ver la lista de símbolos de empresas? Escriba S para sí o N para no: ")
            if opcion_lista.upper() == 'S':
                print("Lista de símbolos de empresas:")
                for empresa in empresas.values():
                    print("- " + empresa['simbolo'])
            simbolo_empresa = input("Escriba el símbolo de la empresa que desea buscar: ")
            empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['simbolo'] == simbolo_empresa.upper()]
            if len(empresas_encontradas) == 0:
                print("No se encontró la empresa.")
            else:
                print("Puedes encontrar información financiera de la empresa en el siguiente link:", empresas_encontradas[0])
                empresa_encontrada = empresas_encontradas[0]
                datos_empresa = empresas[empresa_encontrada]
                numero_empresa= empresas_encontradas[0]
                numero_empresa = 'empresa' + str(contador)
                datos_empresa_seleccionada = {'nombre': datos_empresa['nombre'], 'simbolo': datos_empresa['simbolo'], 'link':empresas_encontradas[0]}
                empresas_seleccionadas[numero_empresa] = datos_empresa_seleccionada
                contador += 1

        else:
            print("Opción no válida.")

        opcion_continuar = input("¿Desea continuar con la búsqueda? Escriba S para sí o N para no: ")
        if opcion_continuar.upper() == 'N':
            print("\n")
            print("Estas son las empresas seleccionadas:")
            print(empresas_seleccionadas)
            return empresas_seleccionadas
            break
            
#Ejecutar la función de búsqueda y guardarla
empresas_seleccionadas= buscar_empresas()

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAs PERUANAs DE LA BVL ########

from selenium import webdriver
from selenium.webdriver import ActionChains
from csv import writer
import time
import openpyxl
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import datetime

enlaces = [empresa['link'] for empresa in empresas_seleccionadas.values()]
nombres = [empresa['nombre'] for empresa in empresas_seleccionadas.values()] #VOLCAN COMPAÑIA MINERA S.A.A. #ALICORP S.A.A.
#Año_actual= datetime.datetime.now().year #Comienza desde 2023

# Crea un objeto de libro de trabajo
#libro = openpyxl.Workbook()

for i, url in enumerate(enlaces):
    driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe")
    driver.get(url)
    driver.maximize_window()
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, 525);")
    
    Año_actual= datetime.datetime.now().year #Comienza desde 2023
    
    actions = ActionChains(driver)
    boton_cookie=driver.find_element(By.XPATH, '/html/body/bvl-root/bvl-main-layout/bvl-footer/footer/bvl-cookies-banner/div/div/div/div[2]/button')
    boton_cookie.click()
    time.sleep(2)
    
    #Seleccionar el Periodo
    time.sleep(3)
    periodo = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[2]/div/div[1]')
    time.sleep(2)
    anual = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[2]/div/div[2]/div[2]')
    time.sleep(3)
    actions = ActionChains(driver)
    actions.move_to_element(periodo).move_to_element(anual).click().perform()
    
    #Mover el cursor a filtro Año    
    año = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[1]')
    time.sleep(2)
    
    libro = openpyxl.Workbook()
    
    #Se pregunta la cantidad de años que desea extraer, Ejm: 2018-2020 es 3
    while True:
        try:
            numero_años = int(input("Ingrese el número de años que desea obtener: "))
            break
        except ValueError:
            print("Número inválido. Por favor, ingrese un número entero válido.")   
            
    verificador=numero_años
    #print("verificador inicial es ", verificador)
    
    año_actual=1 #1 es el año más actual, 2 el año anterior y así sucesivamente
    
    while verificador>0:
    #for año in range(1, numero_años+1):     
        #print(existe_informacion)
            
        año = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[1]')
        time.sleep(2)
        
        #for año in range(1, numero_años)
        
        #Seleccionamos el año
        año_elegido = driver.find_element(By.XPATH, "//*[@id='page-view']/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[2]/div["+str(año_actual)+"]")
        time.sleep(2)
        
        #Siguiente año que se revisará
        #año_actual = año_actual+1
        
        #Seleccionando filtro año
        if año_actual==1:
            actions.move_to_element(año).click()
            time.sleep(2) 
        else:
            actions.move_to_element(año).move_to_element(año_elegido).click().perform()
            time.sleep(3)
            
        #print(año_actual)    
        #Siguiente año que se revisará
        año_actual = año_actual + 1
        #print(año_actual)         
        
        try:
            existe_informacion =driver.find_element(By.XPATH, "//*[@id='page-view']/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-section-header/header/div[1]/h1")             
        except NoSuchElementException:
            existe_informacion = None
        
        #print("La informacion es ", existe_informacion)   
                
        if existe_informacion is not None:
            
            #Abrimos un libro de Excel donde se almacenará la información
            #libro = openpyxl.Workbook()
            #libro.close()
            
            #Contador que ayuda a verificar la cantidad de años extraídos
            verificador -= 1
            #print("Verificador es:", verificador)
            #Año_actual = Año_actual-1
            
            #Seleccionar Estado Financiero:
            #Elegimos los estados principales (si desea otra información, puede automatizar o cambiar valores)
            Estado_Situacion_Financiera = '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[1]/td[1]/a'
            Estado_Resultados='//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[2]/td[1]/a'
            Estado_Flujos_Efectivo='//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[5]/td[1]/a'
            
            #Almacenamos los estados financieros seleccionados
            Estados_Financieros={Estado_Situacion_Financiera:'SituacionFinanciera' , 
                                 Estado_Resultados:'Resultados' , 
                                 Estado_Flujos_Efectivo:'FlujoEfectivo' 
                                }
            
            #Acá comienza lo diferente, la reestructuración del código

            #Finalmente obtenemos la información de cada estado
            for estados, nombre_estado in Estados_Financieros.items():
                #Seleccionar el Estado Financiero
                click_button=driver.find_element(By.XPATH, estados)
                time.sleep(3)
                click_button.click()
                time.sleep(2)

                #Expandir la tabla
                expand_button = driver.find_element(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[1]/div[2]/button/i')
                expand_button.click()

                #Obteniendo el número de filas y columnas de la tabla
                time.sleep(1)
                rows = len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr'))
                col =  len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td'))
                time.sleep(2)
                #print(rows) #77
                #print(col) #4
                
                
                # Crea una nueva hoja con el nombre de la empresa y el año correspondiente
                hoja_nombre = nombre_estado + str(Año_actual)
                #Año_actual = Año_actual-1
                hoja = libro.create_sheet(hoja_nombre)
                encabezado = driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td')
                header = [elem.text for elem in encabezado]
                hoja.append(header)

                data=[]
                fila=[]

                rows = len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr'))
                col =  len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td'))
                time.sleep(2)
                for n in range(2, rows+1):
                    fila = []
                    for b in range(1, col+1):
                        dato = driver.find_element(By.XPATH, "/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr["+str(n)+"]/td["+str(b)+"]")
                        #time.sleep(1)
                        #fila.append(dato.text)
                        #fila.append(dato.get_attribute("textContent"))
                        fila.append(dato.get_attribute("innerHTML"))
                        
                    hoja.append(fila)
                
                #Botón cerrar tabla
                close_button = driver.find_element(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[1]/button')
                close_button.click()
                
                fila=[]
                time.sleep(2)

                # Guarda el archivo de Excel con el nombre de la empresa
                #archivo_excel = nombres[i] + ".xlsx"
                #libro.save(archivo_excel)
                
            Año_actual = Año_actual-1
     
        else:  
            Año_actual = Año_actual-1
            print()
        #print("numero es", numero_años)
            continue
            
        archivo_excel = nombres[i] + ".xlsx"
        libro.save(archivo_excel)
        #libro.close()
        
    # Cerrar el controlador de Selenium
    driver.quit()